In [ ]:
import pandas as pd
import seaborn as sns
import glob
import os
import sys
from sklearn.linear_model import LinearRegression
from sklearn import datasets
import pickle
import joblib
import mariadb
import numpy as np
from sklearn import linear_model

In [ ]:
df2 = pd.read_excel('C:/YCL/PYDATAexam/df_screen.xlsx')

In [ ]:
# 단순회귀분석 모형 객체 생성
lr = LinearRegression()   

# train data를 가지고 모형 학습
lr.fit(X_train, y_train)

# 학습을 마친 모형에 test data를 적용하여 결정계수(R-제곱) 계산
r_square = lr.score(X_test, y_test)
print('R제곱근 : ', r_square)

# 회귀식의 기울기
print('X 변수의 계수 a: ', lr.coef_)

# 회귀식의 y절편
print('상수항 b', lr.intercept_)

In [ ]:
import matplotlib.pyplot as plt
y_hat = lr.predict(X_test)

plt.figure(figsize=(10, 5))
ax1 = sns.kdeplot(y_test, label="y_test")
ax2 = sns.kdeplot(y_hat, label="y_hat", ax=ax1)
plt.legend()
plt.show()
plt.close()

In [ ]:
#모델파일 pkl파일로 저장하기
lr.fit(X_train, y_train)
saved_model = pickle.dumps(lr)

# Load the pickled model
clf_from_pickle = pickle.loads(saved_model)

# Use the loaded pickled model to make predictions
clf_from_pickle.predict(X_train)

joblib.dump(lr, 'filename.pkl')

In [ ]:
clf_from_joblib = joblib.load('filename.pkl') 
clf_from_joblib.predict(X)

In [ ]:
# SQL서버에서 분석할 DB셋 가져오기
try:
    conn = mariadb.connect(
        user="유저이름",
        password="비밀번호",
        host="서버주소",
        port=포트번호,
        database="bigdata"
    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)
cur = conn.cursor()
cur.execute('select * from testing')
res = cur.fetchall()
data = pd.DataFrame.from_records(res)
df4 = pd.DataFrame(data) 
df4.rename(columns= {0:'sex', 1:'age', 2:'age_group', 3:'BO1_2', 4:'BO3_01', 5:'BE3_76', 6:'BE3_78', 7:'BE3_86', 8:'BE3_88', 9:'BE8_1', 10:'BE5_1', 11:'BE9', 12:'HE_ht', 13:'HE_BMI'}, inplace=True)

In [ ]:
df5=df4[['sex','age_group','BE3_76','BE3_78','BE3_86','BE3_88','BE9','HE_BMI']]
clf_from_joblib = joblib.load('filename.pkl') 
clf_from_joblib.predict(df5)
df4['LOSS']=clf_from_joblib.predict(df5) #LOSS가 체중 감소량 예측치
df4['TIME'] = df4['BE3_78'] + df4['BE3_88'] #전체 운동시간
df4['EX_HIGH'] = (df4['BE3_78']/df4['TIME'])*100 #웨이트(고강도) 운동시간
df4['EX_MID'] = (df4['BE3_88']/df4['TIME'])*100#유산소(중강도) 운동시간

In [ ]:
def lin_regress(data, yvar, xvars):

    # output, input변수들
    Y = data[yvar]
    X = data[xvars]
    
    # 선형회귀분석 모델 세팅
    linreg = linear_model.LinearRegression()

    # 모델 피팅
    model = linreg.fit(X, Y)


    # 모델값 저장
    intercept = model.intercept_
    coef = model.coef_
    result = [intercept, coef]

    return result

# 그룹 기준 만들기
grouped = df4.groupby('age_group')

# 그룹변수를 모델에 적용해서 분석
lin_reg_coef = grouped.apply(lin_regress, 'LOSS', [ 'sex','BE3_76','BE3_78','BE3_86','BE3_88','BE9','HE_BMI'])